<a href="https://colab.research.google.com/github/achillesposiedon/GAN/blob/master/GANArtificialMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import keras

from tensorflow.python.keras.layers import Dense, Dropout, Input
from tensorflow.python.keras.models import Model,Sequential
from tensorflow.python.keras.datasets import mnist

from tqdm import tqdm

from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam


Using TensorFlow backend.


In [0]:
def get_data():
    (xtrain, ytrain), (xtest, ytest) = mnist.load_data()
    xtrain = (xtrain.astype(np.float32))/255

    xtrain = xtrain.reshape(60000, 784)
    return (xtrain, ytrain, xtest, ytest)


(xtrain, ytrain,xtest, ytest)=get_data()

In [0]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

In [0]:
def generatornet():
    generator=Sequential()
    generator.add(Dense(units=256,input_dim=100))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=512))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(units=784, activation='sigmoid'))
    
    generator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return generator

gen=generatornet()
gen.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               25856     
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              525312    
_________________________________________________________________
leaky_re_lu_2

In [0]:
def discriminatornet():
    discriminator=Sequential()
    
    discriminator.add(Dense(units=1024,input_dim=784))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(units=1024))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    
    discriminator.add(Dense(units=512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
       
    discriminator.add(Dense(units=256))
    discriminator.add(LeakyReLU(0.2))
    
    discriminator.add(Dense(units=1, activation='sigmoid'))
    
    discriminator.compile(loss='binary_crossentropy', optimizer=adam_optimizer())
    return discriminator
disc =discriminatornet()
disc.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              803840    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)              

In [0]:
def GAN(discriminator, generator):
    discriminator.trainable=False
    noiseinput = Input(shape=(100,))
    x = generator(noiseinput)
    gan_output= discriminator(x)
    gan= Model(inputs=noiseinput, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan
gan = GAN(disc,gen)
gan.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential (Sequential)      (None, 784)               2535952   
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 2509825   
Total params: 5,045,777
Trainable params: 2,535,952
Non-trainable params: 2,509,825
_________________________________________________________________


In [0]:
def plot_img(epoch, generator, examples=100, dim=(10,10), figsize=(10,10)):
    noise= np.random.normal(loc=0, scale=1, size=[examples, 100])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(100,28,28)
    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image %d.png' %epoch)

In [0]:
def training(epochs=1, batch_size=128):
    
    #Loading the data
    (X_train, y_train, X_test, y_test) = get_data()
    batch_count = X_train.shape[0] / batch_size
    
    # Creating GAN
    generator= generatornet()
    discriminator= discriminatornet()
    gan = GAN(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        
        for _ in tqdm(range(batch_size)):

          
          noise= np.random.normal(0,1, [batch_size, 100])
          generated_images = generator.predict(noise)
            
          image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            
          X= np.concatenate([image_batch, generated_images])
          y_dis=np.zeros(2*batch_size)
          y_dis[:batch_size]=0.9             
          discriminator.trainable=True
          discriminator.train_on_batch(X, y_dis)
            
          
          noise= np.random.normal(0,1, [batch_size, 100])
          y_gen = np.ones(batch_size)
            
          discriminator.trainable=False
          gan.train_on_batch(noise, y_gen)
            
          if e == 1 or e % 20 == 0:
            plot_img(e, generator)
        

In [0]:
training(500,128)

NameError: ignored